In [4]:
import pandas as pd
import pyodbc
from bs4 import BeautifulSoup
import requests
username = 'vigrose'
password = 'Hufflepuff10946'
q = "DSN=eprdods; UID={}; PWD={}".format(username, password)
ODS = pyodbc.connect(q)

In [2]:
iqvia_address = \
    """
    SELECT 
    PE.ME_NUMBER,
    P.ME,
    B.IMS_ORG_ID,
    B.BUSINESS_NAME,
    B.PHYSICAL_ADDR_1,
    B.PHYSICAL_ADDR_2,
    B.PHYSICAL_STATE,
    B.PHYSICAL_ZIP,
    B.PHYSICAL_CITY,
    A.AFFIL_RANK
    FROM
    ODS.ODS_PPD_FILE PE, ODS.ODS_IMS_BUSINESS B, ODS.SAS_ODS_IMS_PROVIDER_BEST_AFFIL A, ODS.ODS_IMS_PROFESSIONAL P 
    WHERE
    SUBSTR(PE.ME_NUMBER,1,10)=P.ME
    AND
    P.PROFESSIONAL_ID=A.PROFESSIONAL_ID
    AND
    A.IMS_ORG_ID=B.IMS_ORG_ID
    AND
    PE.PENDINGDEAD_IND IS NULL
    AND
    P.CURRENT_BATCH_FLAG='Y'
    AND
    A.CURRENT_BATCH_FLAG='Y'
    AND
    B.CURRENT_BATCH_FLAG='Y'
    """

In [5]:
def fix_me(me_list):
    '''Add leading zeroes to ME numbers'''
    nums = []
    for num in me_list:
        num = str(num)
        num = num.replace('.0', '')
        if len(num) == 10:
            num = '0' + num
        elif len(num) == 9:
            num = '00' + num
        elif len(num) == 8:
            num = '000' + num
        nums.append(num)
    return nums

In [3]:
iqvia = pd.read_sql(con=ODS, sql=iqvia_address)

In [6]:
amb = pd.read_csv('../../Data/Sales_Ops/with_me.csv')
amb['ME']=fix_me(amb.ME)
ppd = pd.read_csv('../../Data/PPD/ppd_data_20200718.csv')
ppd['ME']=fix_me(ppd.ME)

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (13,20,42,49,56,57,72) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [24]:
pd.merge(amb, iqvia, left_on='ME', right_on='ME_NUMBER', how='left').to_csv('../../Data/Sales_Ops/Iqvia_Add.csv', index=False)

In [7]:
other = pd.read_excel('../../Data/Sales_Ops/additional.xlsx')
other['ME_NUMBER']=fix_me(other.MED_EDU_NBR)

In [8]:
zz = pd.merge(other, iqvia, on='ME_NUMBER', how='left')

In [9]:
match = zz[zz.IMS_ORG_ID.isna()==False][['Email ','ME_NUMBER','BUSINESS_NAME','MED_EDU_NBR']].rename(columns={'BUSINESS_NAME':'ORGANIZATION'})

In [10]:
STUDENT = pd.read_csv('../../Data/Sales_Ops/DBL_STUDENT_FILE_FOR_VICTORIA_UPDATED_MEDSCHOOL.txt',sep="|")
STUDENT['ME']=fix_me(STUDENT.ME)

In [11]:
stud = pd.merge(other, STUDENT, left_on='ME_NUMBER', right_on='ME')[['Email ','ME_NUMBER','MEDSCHOOL_NAME','MED_EDU_NBR']].rename(columns={'MEDSCHOOL_NAME':'ORGANIZATION'})

In [12]:
so_far = pd.concat([stud,match])

In [13]:
unmatched = other[other['Email '].isin(so_far['Email '])==False]

In [27]:
extras = pd.merge(unmatched, ppd, left_on='ME_NUMBER',right_on='ME', how='left')

In [25]:
unmatched[unmatched['Email '].isin(extras['Email '])==False].to_excel('../../Data/Sales_Ops/Missing_MEs.xlsx', index=False)

In [118]:
extras.to_csv('../../Data/Sales_Ops/missing.csv', index=False)

In [16]:
unmatched['em'] = [x.split('@')[1] for x in unmatched['Email ']]
e_list = unmatched.drop_duplicates('em')['em']

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [35]:
dict_list =[]
for link in e_list:
    org = 'None'
    if 'mail' in link:
        new_link = link.split('mail.')[1]
    else:
        new_link = link
    med_url = 'http://'+new_link
    try:
        response = requests.get(med_url)
        try:
            soup = BeautifulSoup(response.content, 'html.parser')
            print(soup.title.text)
            org = soup.title.text
            new_dict = {'email': link, 'org': org}
            dict_list.append(new_dict)
        except:
            print('no title')
    except:
        print('URL did not work')
    

Michigan State University
SUNY Downstate Health Sciences University

	Albert Einstein College of Medicine | Medical Education | Biomedical Research

403 - Forbidden: Access is denied.
Campbell University | Buies Creek, NC | Leading With Purpose

	Healthcare Education & Research Innovation Solutions

UT Health San Antonio
Access Denied
URL did not work
URL did not work
University of Wisconsin–Madison
Home - Eastern Virginia Medical School (EVMS), Norfolk, Hampton Roads
Oakland University
Washington University in St. Louis
Duke University
Home | UConn Health

	University of Oklahoma Health Sciences Center

Home | University Of Cincinnati
Welcome to the University at Buffalo - University at Buffalo
URL did not work
URL did not work
Saint Louis University : SLU
Home - Rocky Vista University
Home | University of Pittsburgh
Icahn School of Medicine at Mount Sinai - New York City |  Icahn School of Medicine
Columbia University Irving Medical Center
Home | Medical College of Wisconsin
Johns Ho

In [60]:
em_org = pd.DataFrame(dict_list)

In [21]:
pd.merge(unmatched, em_org, left_on='em', right_on='email')

,Salutation,First Name,Last Name,Position/Title,Email,Business Phone,Mobile Phone,Street Address,City,State,...,Import Source,Import Date,ADDR_1,ADDR_2,ADDR_3,MED_EDU_NBR,ME_NUMBER,em,email,org
0,Dr.,Rohit,Abraham,NaN,rohit@msu.edu,NaN,NaN,NaN,Boston,MA,...,Ambassador list,NaN,614 Massachusetts Ave,Apt 14,NaN,2512180356,02512180356,msu.edu,msu.edu,Michigan State University
1,Ms.,Briana,Calderon,NaN,calder58@msu.edu,NaN,NaN,NaN,Grand Rapids,MI,...,Ambassador list,NaN,345 Logan St SE Apt 1A,NaN,NaN,2512200829,02512200829,msu.edu,msu.edu,Michigan State University
2,Ms.,Diana,Chen,NaN,chendia3@msu.edu,NaN,NaN,NaN,Sacramento,CA,...,Ambassador list,NaN,8200 Cinnamon Bar Ct,NaN,NaN,2512180496,02512180496,msu.edu,msu.edu,Michigan State University
3,Ms.,Katayoon,Sabetsarvestani,NaN,sabetsar@msu.edu,NaN,NaN,NaN,East Lansing,MI,...,Ambassador list,NaN,1376 E Grand River Ave,Apt 20,NaN,2512200268,02512200268,msu.edu,msu.edu,Michigan State University
4,Mr.,Scott,Abrahamson,NaN,scott.abrahamson@downstate.edu,NaN,NaN,NaN,Melville,NY,...,Ambassador list,NaN,18 Jagger Ct,NaN,NaN,3508202449,03508202449,downstate.edu,downstate.edu,SUNY Downstate Health Sciences University
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
312,Dr.,Kamalpreet,Uppal,NaN,uppals@mail.amc.edu,NaN,NaN,NaN,Albany,NY,...,Ambassador list,NaN,100 Union Dr,Apt 440,NaN,3503191236,03503191236,mail.amc.edu,mail.amc.edu,Albany Medical Center
313,Ms.,Nishma,Valikodath,NaN,nishmav@umich.edu,NaN,NaN,NaN,Ann Arbor,MI,...,Ambassador list,NaN,936 Wall St,Apt 5,NaN,2501202196,02501202196,umich.edu,umich.edu,University of Michigan
314,Ms.,Manna,Varghese,NaN,mmvxb2@mail.umkc.edu,NaN,NaN,NaN,Lees Summit,MO,...,Ambassador list,NaN,117 NW Whitman Dr,NaN,NaN,2846200391,02846200391,mail.umkc.edu,mail.umkc.edu,\n\t\t\tHome | University of Missouri - Kansas...
315,Dr.,Dannen,Wright,NaN,dannen.wright@hsc.utah.edu,NaN,NaN,NaN,Eagle,ID,...,Ambassador list,NaN,2904 W Newbury Ct,NaN,NaN,4901190050,04901190050,hsc.utah.edu,hsc.utah.edu,\n\t\t\tHome | University of Missouri - Kansas...


In [22]:
len(unmatched)

317

In [26]:
extras['POLO_MAILING_LINE_1']

0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
      ... 
241    NaN
242    NaN
243    NaN
244    NaN
245    NaN
Name: POLO_MAILING_LINE_1, Length: 246, dtype: object

In [28]:
extras['email'] = [x.split('@')[1] for x in extras['Email ']]

In [61]:
meyo = pd.merge(extras, em_org, on='email', how='left')

In [62]:
meyo = meyo.fillna("none")

In [63]:
len(meyo)

317

In [64]:
meyo = meyo.rename(columns={'Email ': 'EMAIL'})

In [65]:
dict_list_2 = []
for row in meyo.itertuples():
    if row.POLO_MAILING_LINE_1!='none':
        org = row.POLO_MAILING_LINE_1
    elif row.POLO_MAILING_LINE_2!='none':
        org = row.POLO_MAILING_LINE_2
    else:
        org = row.org
    new_dict = {'EMAIL': row.EMAIL, 'ORGANIZATION':org}
    dict_list_2.append(new_dict)

In [66]:
gah = pd.DataFrame(dict_list_2)

In [67]:
pd.merge(gah, meyo, left_on= 'me', right_on='ME').drop_duplicates(['ME','ORGANIZATION']).to_csv('../../Data/Sales_Ops/Work.csv', index=False)

KeyError: 'me'

In [48]:
len(meyo)

317

In [68]:
pd.merge(gah, meyo, on='EMAIL').drop_duplicates(['EMAIL','ORGANIZATION']).to_csv('../../Data/Sales_Ops/Work.csv', index=False)

In [59]:
em_org

,email,org
0,msu.edu,Michigan State University
1,downstate.edu,SUNY Downstate Health Sciences University
2,mail.einstein.yu.edu,\r\n\tAlbert Einstein College of Medicine | Me...
3,mayo.edu,403 - Forbidden: Access is denied.
4,email.campbell.edu,"Campbell University | Buies Creek, NC | Leadin..."
...,...,...
109,mail.amc.edu,Albany Medical Center
110,umich.edu,University of Michigan
111,mail.umkc.edu,\n\t\t\tHome | University of Missouri - Kansas...
112,hsc.utah.edu,\n\t\t\tHome | University of Missouri - Kansas...


In [76]:
EXTRA_MATCH = pd.read_csv('../../Data/Sales_Ops/Work.csv')[['EMAIL','ORGANIZATION']]

In [73]:
other_match = pd.read_excel('../../Data/Sales_Ops/other_match.xlsx')

In [77]:
so_far

,Email,ME_NUMBER,ORGANIZATION,MED_EDU_NBR
0,oluwatosin.adebiyi@ucdenver.edu,00702211549,University of Colorado School of Medicine,702211549
1,afrinant@msu.edu,02512220471,Michigan State University College of Human Med...,2512220471
2,bukky.ajagbe@gmail.com,04775200087,Lincoln Memorial University Debusk College of ...,4775200087
3,saallen@mail.einstein.yu.edu,03546210307,Albert Einstein College of Medicine of Yeshiva...,3546210307
4,ymamer01@louisville.edu,02002210803,University of Louisville School of Medicine,2002210803
...,...,...,...,...
832,sophiayang88@gmail.com,00515170146,SANTA CLARA VALLEY MEDICAL CENTER,515170146
834,annalyap@gmail.com,00512180868,RONALD REAGAN UNIVERSITY OF CALIFORNIA LOS ANG...,512180868
836,mybarra@phrma.org,01002080873,MEDSTAR GEORGETOWN UNIVERSITY HOSPITAL,1002080873
838,yeltoncj@gmail.com,04504200456,DARTMOUTH-HITCHCOCK MEDICAL CENTER,4504200456


In [78]:
EXTRA_MATCH

,EMAIL,ORGANIZATION
0,pollock.aaron@knights.ucf.edu,NaN
1,aranderson4@wisc.edu,University of Wisconsin-Madison
2,solom110@umn.edu,University of Minnesota Twin Cities
3,abigailmcrea@gmail.com,NaN
4,aeprescott1@gmail.com,NaN
...,...,...
311,muhlenberg@aol.com,tower health medical group
312,wlawsonpsy@gmail.com,NaN
313,yparekh@mail.einstein.yu.edu,Albert Einstein College of Medicine
314,bahr.zachary@gmail.com,NaN


In [80]:
better_match = pd.merge(other_match, EXTRA_MATCH, left_on='email', right_on='EMAIL')[['email','org']]

In [82]:
extra_match = EXTRA_MATCH[EXTRA_MATCH.EMAIL.isin(better_match.email)==False]

In [84]:
better_match = better_match.rename(columns={'email':'EMAIL','org':'ORGANIZATION'})

In [85]:
extra_match = extra_match.rename(columns={'email':'EMAIL','org':'ORGANIZATION'})

In [91]:
so_far = so_far.rename(columns={'Email ':'EMAIL'})

In [93]:
les_go = pd.concat([better_match, extra_match, so_far[['EMAIL','ORGANIZATION']]])

In [99]:
les_go = les_go.fillna('none')
les_go['ORGANIZATION']= [x.upper() for x in les_go.ORGANIZATION]

In [104]:
pd.merge(other, les_go, left_on='Email ', right_on='EMAIL', how='left').drop_duplicates('Email ').to_csv('../../Data/Sales_Ops/Finally.csv', index=False)

In [110]:
pd.merge(other, les_go, left_on='Email ', right_on='EMAIL', how='left')[647:]

,Salutation,First Name,Last Name,Position/Title,Email,Business Phone,Mobile Phone,Street Address,City,State,...,ACTIVE,Import Source,Import Date,ADDR_1,ADDR_2,ADDR_3,MED_EDU_NBR,ME_NUMBER,EMAIL,ORGANIZATION
647,Dr.,Stephen,Rockower,NaN,drrockower@gmail.com,3.017708e+09,NaN,NaN,Rockville,MD,...,NaN,Ambassador list,NaN,6000 Executive Blvd,Ste 510,NaN,4113751130,04113751130,drrockower@gmail.com,"CAPITOL ORTHOPAEDICS AND REHABILITATION, LLC"
648,Ms.,Hilary,Rogers,NaN,hilary.rogers128@gmail.com,NaN,NaN,NaN,Providence,RI,...,NaN,Ambassador list,NaN,308 Jastram St,NaN,NaN,2407201003,02407201003,hilary.rogers128@gmail.com,SANTA CLARA VALLEY MEDICAL CENTER
649,Dr.,Walter,Rok,NaN,wjrok@cox.net,5.083247e+09,NaN,NaN,Barrington,RI,...,NaN,Ambassador list,NaN,15 Owings Stone Rd,NaN,NaN,3519801255,03519801255,wjrok@cox.net,"PEDIATRIC ASSOCIATES OF FALL RIVER, INC"
650,Dr.,Andrea,Rosado,NaN,rosadoam@mail.uc.edu,NaN,NaN,NaN,Cincinnati,OH,...,NaN,Ambassador list,NaN,1740 E McMillan St,NaN,NaN,3841191004,03841191004,rosadoam@mail.uc.edu,THE CHRIST HOSPITAL
651,Dr.,Mattie,Rosi-Schumacher,NaN,mattiero@buffalo.edu,NaN,NaN,NaN,Hummelstown,PA,...,NaN,Ambassador list,NaN,1827 Gramercy Pl,NaN,NaN,3506190389,03506190389,mattiero@buffalo.edu,PENN STATE MILTON S HERSHEY MEDICAL CENTER
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
845,Mr.,Eric,Zhang,NaN,ejz321@gmail.com,NaN,NaN,NaN,Naperville,IL,...,NaN,Ambassador list,NaN,2051 Steeplebrook Ct,NaN,NaN,1642200468,01642200468,ejz321@gmail.com,NONE
846,Mr.,Jeffrey,Zhao,NaN,jeffrey.zhao1@northwestern.edu,NaN,NaN,NaN,Chicago,IL,...,NaN,Ambassador list,NaN,25 E Superior St Apt 4003,NaN,NaN,1606210520,01606210520,jeffrey.zhao1@northwestern.edu,NORTHWESTERN UNIVERSITY FEINBERG SCHOOL OF MED...
847,Mr.,Dawei,Zhao,NaN,dwzhao@bu.edu,NaN,NaN,NaN,Lexington,MA,...,NaN,Ambassador list,NaN,45 N Hancock St,NaN,NaN,2405210773,02405210773,dwzhao@bu.edu,BOSTON UNIVERSITY SCHOOL OF MEDICINE
848,Ms.,Cheryl,Zogg,NaN,cheryl.zogg@yale.edu,NaN,NaN,NaN,New Haven,CT,...,NaN,Ambassador list,NaN,360 State St,Apt 2916,NaN,801220240,00801220240,cheryl.zogg@yale.edu,NONE


In [111]:
les_go

,EMAIL,ORGANIZATION
0,fonsecacmd@gmail.com,NONE
1,asingh11@luc.edu,UMASS MEMORIAL MEDICAL CENTER UNIVERSITY CAMPUS
2,kmcavoy@mcw.edu,THE MEDICAL COLLEGE OF WISCONSIN
3,amber.gautam@gmail.com,THE BRODY SCHOOL OF MEDICINE AT EAST CAROLINA ...
4,forbesl10@students.ecu.edu,BRODY SCHOOL OF MEDICINE AT EAST CAROLINA UNIV...
...,...,...
832,sophiayang88@gmail.com,SANTA CLARA VALLEY MEDICAL CENTER
834,annalyap@gmail.com,RONALD REAGAN UNIVERSITY OF CALIFORNIA LOS ANG...
836,mybarra@phrma.org,MEDSTAR GEORGETOWN UNIVERSITY HOSPITAL
838,yeltoncj@gmail.com,DARTMOUTH-HITCHCOCK MEDICAL CENTER


In [113]:
so_far[so_far['EMAIL']=='brigittamd@aol.com']

,EMAIL,ME_NUMBER,ORGANIZATION,MED_EDU_NBR
645,brigittamd@aol.com,01720941968,"ROBINSON, BRIGITTA, OFFICE",1720941968
750,brigittamd@aol.com,02846820445,POUDRE VALLEY HEALTH SYSTEM,2846820445
